Code used to power-law fit the data.
Google Drive Folder with data: https://drive.google.com/drive/folders/13KgY7vxar1qEAdFZqI91m9O2sgiC0o7_?usp=share_link

Note: Some codes are file format sensitive (csv/xlsx)

In [ ]:
import os
import glob
import math
import numpy as np
import pandas as pd
import random
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

import scipy
from scipy import stats
from scipy.stats import t
from scipy.optimize import curve_fit
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Add the path

In [ ]:
path = "" #add the path
df = pd.read_excel(path).fillna(0)
df

In [ ]:
def get_rss(yy, ff):
  n_data = len(yy)
  yy_mean = np.mean(yy)

  # Calculate residual and total sum of squares
  residual_sum_squares = 0
  total_sum_squares = 0
  for i in range(n_data):
      residual_sum_squares += (yy[i] - ff[i])**2
      total_sum_squares += (yy[i] - yy_mean)**2

  return residual_sum_squares, total_sum_squares

In [ ]:
def power_law(x, a, b):
  return a * np.power(x, b)


def plot_data(category, path):
  """
  dir: Directory where all files are present

  """
  df = pd.read_excel(path).fillna(0)

  x = df['Total']
  y = df[category]
  

  y = y[y != 0] # Remove zero values
  x = x[y.index] # Select corresponding x values
  logx, logy = np.log(x), np.log(y)

  b, loga = np.polyfit(logx, logy, 1)
  a = np.exp(loga)

  #plt.style.use('seaborn')
  #sns.set_style('darkgrid')

  
  pars, cov = curve_fit(f=power_law, xdata=x, ydata=y, p0=[0, 0], bounds=(-np.inf, np.inf), maxfev = 1000)
  stdevs = np.sqrt(np.diag(cov))
  rss, _ = get_rss(np.log(y.to_numpy()), np.log(power_law(x, *pars).to_numpy()))
  print(rss)

  rss_my = ((y.to_numpy() - power_law(x, *pars).to_numpy()) ** 2).sum()

  #plt.xscale('log')
  #plt.yscale('log')
  #plt.xlim([1e2, 1e5])
  #plt.scatter(x, y)
  #plt.title(category)

  #ax = sns.regplot(logx, logy, ci=95, color='black')
  #plt.xlabel("Total Functional Prtotein annotations")
  #plt.ylabel("Category annotations")
  
  #slope_intercept = np.polyfit(logx, logy, 1)
  #res_power_law = stats.linregress(logx, logy) # Residual for linear fit

  res = stats.linregress(logx, logy)

  # See https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
  tinv = lambda p, df: abs(t.ppf(p/2, df))
  ts = tinv(0.05, len(logx) - 2)
  std_err = res.stderr

  # plt.text(800, 80, f"Slope: {b:.6f} +/- {ts * std_err:.6f}")
  print(f"{category}   {b:.2f} +/- {ts * std_err:.2f}")
  #plt.plot(x, power_law(x, *pars), linestyle='-', linewidth=1, color=None, label=f"Slope: {b:.6f} +/- {ts * std_err:.6f}")
  #plt.legend(loc=0)
  #plt.show()
  return b, ts * std_err, a
 

In [ ]:
slopes = []
slope_errs = []
for category in df.columns[1:-1]:
  slope, slope_err, normalization_constant = plot_data(category, path)
  print(f"Normalization constant: {normalization_constant}")
  slopes.append(slope)
  slope_errs.append(slope_err)

with open("store.csv", "w") as f:
  for slope in slopes:
    f.write(f"{slope}\n")
  for slope_err in slope_errs:
    f.write(f"{slope_err}\n")
plt.savefig('ar', bbox_inches='tight', dpi=1000)  

import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages('multipage.pdf')
plt.savefig(pp, format='pdf')
pp.savefig()
pp.close()
print(b, error, a)